In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import random
import os
import pyproj
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()
from IPython.display import display
import zipfile

# Nombres y Direcciones particulares de cada caso

In [3]:
# Dirección donde se ubican los archivos que se cargarán
path_case='A:/CentrodeEnergia/PLP20230104/OPLP20230104/data/'
# path_case = 'C:/Users/nicol/Documents/CentrodeEnergia/data-set-plexo/'
# path_case='C:/Users/Lenovo/Documents/Work/Practica_II/Plexos/'
# Nombre que tendrá el caso
name_Case='PLP20230104'
# name_Case='PLEXOS'

# Inicio

## Importando Data

In [4]:
remotedesk=False

if remotedesk:
    path_data='C:/Users/Centro/Documents/DataPLP/'
else:
    path_data=path_case

## Barra import

In [6]:
plpbar=pd.read_csv(path_data+'plpbar.csv')
plpbar.columns=["Hidro","time","TipoEtapa","id","BarName","CMgBar","DemBarP","DemBarE","PerBarP","PerBarE","BarRetP","BarRetE"]
plpbar['BarName']=plpbar['BarName'].str.replace(" ","")
plpbar["Hidro"] = plpbar["Hidro"].str.replace(" ", "")

indexbus=plpbar[['id','BarName']].drop_duplicates(keep="first").reset_index(drop=True)

ubibar=pd.read_csv(path_data+'ubibar.csv',sep=';')
ubibar=ubibar.drop('ID',axis=1)
ubibar['LATITUD']=ubibar['LATITUD'].apply(lambda x:x.replace(',','.')).apply(float)
ubibar['LONGITUD']=ubibar['LONGITUD'].apply(lambda x:x.replace(',','.')).apply(float)
ubibar.columns=["BarName","latitud","longitud"]
ubibar['BarName']=ubibar['BarName'].str.replace(" ","")


## Central import

In [7]:
plpcen=pd.read_csv(path_data+'plpcen.csv')
plpcen.columns=["Hidro","time","TipoEtapa","id","CenName","tipo","bus_id","BarName","CenQgen","CenPgen","CenEgen","CenInyP","CenInyE","CenRen","CenCVar","CenCostOp","CenPMax"]
plpcen['CenName']=plpcen["CenName"].str.replace(" ","")
plpcen=plpcen.drop(["CenEgen","CenInyP","CenInyE","CenRen","CenCostOp","CenPMax"],axis=1)
plpcen["Hidro"] = plpcen["Hidro"].str.replace(" ", "")
plpcen['tipo']='otros'

indexcen=plpcen[['id','CenName','tipo','bus_id']].drop_duplicates(keep="first").reset_index(drop=True)
centralsinfo=pd.read_csv(path_data+'centralesinfo.csv',sep=';')
centralsinfo.columns=['id','CenName','type','CVar','effinciency','bus_id','serie_hidro_gen','serie_hidro_ver','min_power','max_power',"VembIn","VembFin","VembMin","VembMax","cotaMínima"]

cols = ['min_power', 'max_power', 'effinciency', 'CVar', 'VembIn', 'VembFin', 'VembMin', 'VembMax', 'cotaMínima']
centralsinfo['CenName'] = centralsinfo["CenName"].str.replace(" ", "")
for col in cols:
    centralsinfo[col] = centralsinfo[col].replace(",", ".", regex=True)


In [8]:
hydric_adicional = pd.read_csv(path_data+'hydric_adicional.csv',sep=";")

tiposcentrales=pd.read_csv(path_data+'centralestype.csv',encoding='latin-1').rename(columns={'cen_name':'CenName'})
typecentrals=indexcen.merge(tiposcentrales,on='CenName')

for x in range(len(indexcen['id'])):
    tipo=typecentrals[typecentrals['CenName']==indexcen['CenName'][x]]['cen_type'].values
    
    if len(tipo)>0:
        plpcen.loc[plpcen['id'] == indexcen['id'][x], 'tipo'] = tipo[0]
    
    

## Lineas import

In [9]:
plplin=pd.read_csv(path_data+'plplin.csv')
# Cambiando los nombres de las columnas
plplin.columns=["Hidro","time","TipoEtapa","id","LinName","bus_a","bus_b","LinFluP","LinFluE","capacity","LinUso","LinPerP","LinPerE","LinPer2P","LinPer2E","LinITP","LinITE"]
plplin['LinName']=plplin['LinName'].str.replace(" ","")
plplin["Hidro"] = plplin["Hidro"].str.replace(" ", "")

indexlin=plplin[['id','LinName',"bus_a","bus_b"]].drop_duplicates(keep="first").reset_index(drop=True)

C:\Users\nicol\AppData\Local\Temp\ipykernel_21796\1961313449.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  plplin=pd.read_csv(path_data+'plplin.csv')


In [10]:
linesinfo=pd.read_csv(path_data+'linesinfo.csv',sep=';')
linesinfo.columns=["id","LinName","bus_a","bus_b","max_flow_a_b","max_flow_b_a","voltage","r","x","segments","active"]
linesinfo['LinName']=linesinfo['LinName'].str.replace(" ","")
linesinfo['max_flow_a_b']=(linesinfo["max_flow_a_b"].apply(str)).apply(lambda x:x.replace(',','.')).apply(float)
linesinfo['max_flow_b_a']=(linesinfo['max_flow_b_a'].apply(str)).apply(lambda x:x.replace(',','.')).apply(float)
linesinfo['r']=(linesinfo['r'].apply(str)).apply(lambda x:x.replace(',','.')).apply(float)
linesinfo['x']=(linesinfo['x'].apply(str)).apply(lambda x:x.replace(',','.')).apply(float)

linesfinal=indexlin.drop(['id','bus_a','bus_b'],axis=1).merge(linesinfo,on='LinName')
linesfinal['id']=(linesfinal['id']).apply(int)
linesfinal.sort_values("id", inplace=True)

## Reservoirs Import

In [27]:
reservoirs = pd.read_csv(path_data+'plpemb.csv')
reservoirs.rename(columns={'Bloque': 'time', 'EmbNum': 'id', 'EmbNom': 'EmbName'}, inplace=True)
reservoirs['EmbName']=reservoirs['EmbName'].str.replace(" ","")
reservoirs['Hidro']=reservoirs['Hidro'].str.replace(" ","")

indexres = reservoirs[['id','EmbName']].drop_duplicates(keep="first").reset_index(drop=True)

junctionsinfo=centralsinfo[centralsinfo['type'].isin(["E",'S','R'])].reset_index(drop=True)
reservoirsinfo=centralsinfo[centralsinfo['type'].isin(["E"])].reset_index(drop=True)
reservoirsinfo.rename(columns={'CenName':'EmbName'}, inplace=True)

for i, emb_name in enumerate(reservoirsinfo['EmbName']):
    if emb_name in indexres['EmbName'].values:
        idx = indexres.index[indexres['EmbName'] == emb_name][0]
        reservoirsinfo.at[i, 'id'] = indexres.at[idx, 'id']



## Indhor import

In [13]:
indhor = pd.read_csv(path_data+'indhor.csv',encoding='latin-1')
# indhor = pd.read_csv(path_data+'indhor.csv')

# Creando directorios

In [14]:
# Nombre data
namedata=name_Case
electricTopology=namedata+'/Topology/Electric'
hydricTopology=namedata+'/Topology/Hydric'

os.makedirs(electricTopology,exist_ok=True)
os.makedirs(hydricTopology,exist_ok=True)


hidrolist=plpbar['Hidro'].unique()
busscenariolist=[]
centralscenariolist=[]
linescenariolist=[]
reservoirscenariolist=[]
for hidronum in range(len(hidrolist)):
	# Creamos los directorios
	busscenario= namedata+f'/Scenarios/{hidronum+1}/Bus'
	centralscenario=namedata+f'/Scenarios/{hidronum+1}/Centrals'
	linescenario=namedata+f'/Scenarios/{hidronum+1}/Lines'
	reservoirscenario=namedata+f'/Scenarios/{hidronum+1}/Reservoirs'

	os.makedirs(busscenario,exist_ok=True)
	busscenariolist.append(busscenario)

	os.makedirs(centralscenario,exist_ok=True)
	centralscenariolist.append(centralscenario)

	os.makedirs(linescenario,exist_ok=True)
	linescenariolist.append(linescenario)

	os.makedirs(reservoirscenario,exist_ok=True)
	reservoirscenariolist.append(reservoirscenario)

marginal_cost_path=namedata+f'/Scenarios/Marginal_cost_percentil'
line_flow_percentil_path=namedata+f'/Scenarios/Flow_Line_percentil'
generation_sistem_path=namedata+f'/Scenarios/Generation_system'
os.makedirs(marginal_cost_path,exist_ok=True)
os.makedirs(line_flow_percentil_path,exist_ok=True)
os.makedirs(generation_sistem_path,exist_ok=True)
hydrofile = [x for x in range(1,len(hidrolist)+1)]

with open( namedata+'/Scenarios/hydrologies.json', 'w') as f:
  json.dump(hydrofile, f)



# Variables indicadoras de cantidades

In [16]:
# Número de horas de bloques temporales del proyecto
time=plplin['time'].max()

# Número de barras
nbus=len(indexbus['id'])
lbus=list(indexbus['id'])

# Número de generadores
ngen=len(indexcen['id'])

# Número de lineas
nlin=len(indexlin['id'])

# Número de Reservoirs
nres = len(reservoirs['EmbName'].unique())

# Mixed de lineas previcionales

In [7]:
filtered_lines = linesfinal[linesfinal.duplicated(subset=['bus_a', 'bus_b'], keep=False)]

groups = filtered_lines.groupby(['bus_a', 'bus_b'])['id']

parallel_lines = []
for _, group in groups:
    parallel_lines.append(group.values.tolist())

print(parallel_lines)     

[[422, 423], [256, 257], [221, 387], [388, 389], [407, 407], [414, 415], [433, 434, 435, 436, 437, 438, 439, 440, 441]]


In [32]:
linesfinal[linesfinal['id']>406].head()

,LinName,id,bus_a,bus_b,max_flow_a_b,max_flow_b_a,voltage,r,x,segments,active
406,NuevaLagunas220->Lagunas220- mixed,407,296,191,1000.0,1000.0,220,0.045773,0.688750,3,1
407,NuevaLagunas220->Lagunas220- mixed,407,296,191,1000.0,1000.0,220,0.045773,0.688750,3,1
408,Tarapaca220->NuevaLagunas220,408,215,296,183.0,183.0,220,1.247679,9.935893,3,0
409,Cumbre500->Llullaillaco500,409,233,297,1780.0,1410.0,500,1.736070,4.722890,3,1
410,Llullaillaco500->Parinas500,410,297,260,1780.0,1410.0,500,0.183750,2.336250,3,1


In [8]:
lnesfinal_aux = linesfinal.copy()
for sublist in parallel_lines:
    first_element = sublist[0]
    suma_maxab=0
    suma_maxba=0
    linesfinal.loc[linesfinal['id'] == first_element, 'LinName'] += "- mixed"
    for element in sublist:
        if linesfinal.loc[linesfinal['id'] == element, 'active'].values[0] == 1: 
            suma_maxab+=linesfinal.loc[linesfinal['id'] == element, 'max_flow_a_b'].values[0]
            suma_maxba+=linesfinal.loc[linesfinal['id'] == element, 'max_flow_b_a'].values[0]
            linesfinal.loc[linesfinal['id'] == element, 'active'] = 0
    linesfinal.loc[linesfinal['id'] == first_element, 'active'] = 1
    linesfinal.loc[linesfinal['id'] == first_element, 'max_flow_a_b']=suma_maxab
    linesfinal.loc[linesfinal['id'] == first_element, 'max_flow_b_a']=suma_maxba

In [13]:
# Crear un array con todos los primeros elementos de las listas en 'parallel_lines'
first_elements = np.array([x[0] for x in parallel_lines])
plplin_copy = plplin.copy()
# Añadir "- mixed" al final de los valores en 'LinName' donde 'id' hace match con el primer valor de cada lista en 'parallel_lines'
plplin_copy['LinName'] = np.where(plplin_copy['id'].isin(first_elements), plplin_copy['LinName'] + '- mixed', plplin_copy['LinName'])

plplin_copy = pd.merge(plplin_copy, lnesfinal_aux[['id', 'active']], on='id', how='left')

# Crear un nuevo dataframe donde cada fila es una lista en 'parallel_lines'
parallel_df = plplin_copy[plplin_copy['id'].isin([item for sublist in parallel_lines for item in sublist])].copy()

# Creamos una columna "parallel_id" que tenga el primer id de la línea paralela correspondiente para cada fila.
parallel_dict = {id_par: par[0] for par in parallel_lines for id_par in par}
parallel_df['parallel_id'] = parallel_df['id'].map(parallel_dict)

# Ahora podemos agrupar por 'Hidro', 'time' y 'parallel_id', y sumar 'LinFluP' y 'capacity' dentro de cada grupo.
grouped_df = parallel_df[parallel_df['active'] == 1].groupby(['Hidro', 'time', 'parallel_id'])[['LinFluP', 'capacity']].sum().reset_index()

# Primero, fusionamos 'parallel_df' con 'grouped_df'.
result = pd.merge(parallel_df, grouped_df, on=['Hidro', 'time', 'parallel_id'], how='left', suffixes=('', '_sum'))

# Ahora, reemplazamos los valores de 'LinFluP' y 'capacity' con los de 'LinFluP_sum' y 'capacity_sum' solo para las filas donde 'id' es igual a 'parallel_id'.
result.loc[result['id'] == result['parallel_id'], 'LinFluP'] = result['LinFluP_sum']
result.loc[result['id'] == result['parallel_id'], 'capacity'] = result['capacity_sum']

# Finalmente, eliminamos las columnas 'LinFluP_sum' y 'capacity_sum' ya que no las necesitamos más.
result = result.drop(columns=['LinFluP_sum', 'capacity_sum'])

# Si también quieres eliminar la columna 'parallel_id', puedes hacerlo así:
result = result.drop(columns=['parallel_id'])

# Asignar los valores calculados en 'result' a las filas correspondientes en 'plplin_copy'
plplin_copy.set_index(['Hidro', 'time', 'id'], inplace=True)
result.set_index(['Hidro', 'time', 'id'], inplace=True)

# plplin_copy.update(result[['LinFluP', 'capacity']])

# # Restablecer el índice
# plplin_copy.reset_index(inplace=True)

# plplin = plplin_copy

In [17]:
plplin_copy.tail()

TipoEtapa                   LinName  bus_a  bus_b  LinFluP  \
Hidro time id                                                                   
MEDIA 2756 441  10 Bloques    Candidato1->Candidato2_9    306    307      0.0   
      2757 441  10 Bloques    Candidato1->Candidato2_9    306    307      0.0   
      2758 441  10 Bloques    Candidato1->Candidato2_9    306    307      0.0   
      2759 441  10 Bloques    Candidato1->Candidato2_9    306    307      0.0   
      2760 441  10 Bloques    Candidato1->Candidato2_9    306    307      0.0   

                LinFluE  capacity  LinUso  LinPerP  LinPerE  LinPer2P  \
Hidro time id                                                           
MEDIA 2756 441      0.0       0.0     0.0      0.0      0.0       0.0   
      2757 441      0.0       0.0     0.0      0.0      0.0       0.0   
      2758 441      0.0       0.0     0.0      0.0      0.0       0.0   
      2759 441      0.0       0.0     0.0      0.0      0.0       0.0   
      2760 441      0.0       0.0     0.0      0.0      0.0       0.0   

                LinPer2E LinITP  LinITE  active  
Hidro time id                                    
MEDIA 2756 441       0.0    0.0     0.0     0.0  
      2757 441       0.0    0.0     0.0     0.0  
      2758 441       0.0    0.0     0.0     0.0  
      2759 441       0.0    0.0     0.0     0.0  
      2760 441       0.0    0.0     0.0     0.0

In [28]:
duplicated_indices = plplin_copy.index.duplicated(keep=False)
a=plplin_copy[duplicated_indices]
pd.set_option('display.multi_sparse', False)

a


,,,TipoEtapa,LinName,bus_a,bus_b,LinFluP,LinFluE,capacity,LinUso,LinPerP,LinPerE,LinPer2P,LinPer2E,LinITP,LinITE,active
Hidro,time,id,,,,,,,,,,,,,,,
Sim1,1,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,1,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,2,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,2,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,3,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MEDIA,2758,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,95.37,3.72,1000.0,9.54,0.03,0.0,0.01,0.0,0.21,0.01,1.0
MEDIA,2759,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,89.64,7.26,1000.0,8.96,0.03,0.0,0.01,0.0,0.06,0.00,1.0
MEDIA,2759,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,89.64,7.26,1000.0,8.96,0.03,0.0,0.01,0.0,0.06,0.00,1.0


In [27]:
duplicated_indices = result.index.duplicated(keep=False)
a=result[duplicated_indices]
pd.set_option('display.multi_sparse', False)

a


,,,TipoEtapa,LinName,bus_a,bus_b,LinFluP,LinFluE,capacity,LinUso,LinPerP,LinPerE,LinPer2P,LinPer2E,LinITP,LinITE,active
Hidro,time,id,,,,,,,,,,,,,,,
Sim1,1,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,1,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,2,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,2,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
Sim1,3,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MEDIA,2758,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,190.74,3.72,2000.0,9.54,0.03,0.0,0.01,0.0,0.21,0.01,1.0
MEDIA,2759,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,179.28,7.26,2000.0,8.96,0.03,0.0,0.01,0.0,0.06,0.00,1.0
MEDIA,2759,407,10 Bloques,NuevaLagunas220->Lagunas220- mixed,296,191,179.28,7.26,2000.0,8.96,0.03,0.0,0.01,0.0,0.06,0.00,1.0


# Función generadora de latitudes y longitudes

In [17]:
def aleatory_direction():
    latitud=-random.uniform(10, 85)
    longitud=-random.uniform(10, 85)
    return latitud,longitud

def LatLon_To_XY(Lat,Lon):
  B = pyproj.Transformer.from_crs(4326,20049) #WGS84->EPSG:20049 (Chile 2021/UTM zone 19S)
  UTMx, UTMy = B.transform(Lat,Lon)
  return UTMx, UTMy

def XY_To_LatLon(x,y):
  B = pyproj.Transformer.from_crs(20049,4326)
  Lat, Lon = B.transform(x,y)
  return Lat, Lon

def valorXY(LatP, LonP, scale):
  A = LatLon_To_XY(LatP, LonP)
  X,Y = A[0]*scale, A[1]*scale
  return Y,X

# Bloques a Fechas

In [47]:
# indhor2=indhor.drop('Hora',axis=1).groupby(['Año','Mes'])
indhor2=indhor.drop('Hora',axis=1).groupby(['Dia','Mes'])

indhorlist=[]
for x in indhor2:
    indhorlist.append([str(x[1]['Bloque'].min()),str(x[1]['Bloque'].max()),str(x[0])])
with open( namedata+'/Scenarios/indhor.json', 'w') as f:
  json.dump(indhorlist, f)

# Generación por Sistema por Hidrología

In [18]:
typegenlist=typecentrals.cen_type.unique()
for i,hydro in enumerate(hidrolist):
    print(hydro+" lista")
    dic_type_gen={}
    auxdf = plpcen[plpcen['Hidro']==hydro]
    auxdf=auxdf.groupby(['tipo','time'])['CenPgen'].sum().reset_index().groupby('tipo')
    for group in auxdf:
        tipo = group[0]
        df_tipo = group[1]
        dic_type_gen[tipo] = [row for row in df_tipo[['time', 'CenPgen']].to_dict(orient='records')]
    
    with open(generation_sistem_path+f'/generation_system_{i+1}.json', 'w') as f:
        json.dump(dic_type_gen, f)

Sim1 lista


# Percentiles Costo Marginal

In [19]:
def percentilCM():
    datos_bar = plpbar[['Hidro', 'time','id', 'BarName', 'CMgBar']]
    lista_bar = datos_bar.BarName.unique()

    i=1
    for barra in lista_bar:
        print(f'Procesando datos de {barra} [{i}/{len(lista_bar)}]')
        data_barraTx = datos_bar.loc[(datos_bar.BarName == barra)]
        idbar=data_barraTx['id'].unique()[0]
        data_barraTx = data_barraTx[~(data_barraTx['Hidro'] == 'MEDIA')]
        Promedio = data_barraTx[['time','CMgBar']]
        xy =Promedio.groupby(['time']).mean()
        
        data_barraTx = data_barraTx.groupby(['time']).agg(perc0=('CMgBar', lambda x: x.quantile(0.0)),
                                                                perc20=(
                                                                    'CMgBar', lambda x: x.quantile(0.2)),
                                                                perc80=(
                                                                    'CMgBar', lambda x: x.quantile(0.8)),
                                                                perc100=('CMgBar', lambda x: x.quantile(1)))

        data_barraTx['promedio'] = xy
        data_barraTx = data_barraTx.assign(name=barra)
        data_barraTx = data_barraTx.assign(id=idbar)
        data_barraTx.reset_index(inplace=True)
        data_barraTx=data_barraTx[['id','time','name','perc0','perc20','perc80','perc100','promedio']]
        data_barraTx.to_json(marginal_cost_path+f"/bus_{idbar}.json",orient='records')
        i=i+1


percentilCM()

Procesando datos de AltoNorte110 [1/214]


Procesando datos de Andes220 [2/214]
Procesando datos de Andes345 [3/214]
Procesando datos de Angamos220 [4/214]
Procesando datos de Antofagasta110 [5/214]
Procesando datos de Arica066 [6/214]
Procesando datos de Atacama220_BP1 [7/214]
Procesando datos de Barriles220 [8/214]
Procesando datos de Cachiyuyal220 [9/214]
Procesando datos de Capricornio110 [10/214]
Procesando datos de Capricornio220 [11/214]
Procesando datos de Cardones110 [12/214]
Procesando datos de Cardones220 [13/214]
Procesando datos de Chacaya220 [14/214]
Procesando datos de Chuquicamata100 [15/214]
Procesando datos de Chuquicamata220 [16/214]
Procesando datos de Cochrane220 [17/214]
Procesando datos de Collahuasi220 [18/214]
Procesando datos de Conchi220 [19/214]
Procesando datos de Condores220 [20/214]
Procesando datos de CPinto220 [21/214]
Procesando datos de Crucero220 [22/214]
Procesando datos de Cumbres500 [23/214]
Procesando datos de DAlmagro110 [24/214]
Procesando datos de DAlmagro220 [25/214]
Procesando datos 

# Percentiles Flujos de Lineas de Transmisión

In [20]:
def percentilFL():
    datos_lineas=plplin[['id','Hidro', 'time', 'LinName', 'LinFluP', 'capacity']]
    lista_lineas = datos_lineas.LinName.unique()
    n_lineas = len(lista_lineas)
    i=1
    for linea in lista_lineas:
        print(f'Procesando datos de {linea} [{i}/{n_lineas}]')
        data_lineaTx = datos_lineas.loc[(datos_lineas.LinName == linea)]
        idlin=data_lineaTx['id'].unique()[0]
        data_lineaTx = data_lineaTx[~(data_lineaTx['Hidro'] == 'MEDIA')]
        fluMax = data_lineaTx[['time','capacity']]
        xy =-fluMax.groupby(['time']).max()
        data_lineaTx = data_lineaTx.groupby(['time']).agg(perc0=('LinFluP', lambda x: x.quantile(0.0)),
                                                                perc20=(
                                                                    'LinFluP', lambda x: x.quantile(0.2)),
                                                                perc80=(
                                                                    'LinFluP', lambda x: x.quantile(0.8)),
                                                                perc100=('LinFluP', lambda x: x.quantile(1)))

        data_lineaTx['Min'] = xy
        data_lineaTx['Max'] = -xy
        i = i+1
        data_lineaTx.reset_index(inplace=True)
        data_lineaTx = data_lineaTx.assign(id=idlin)
        data_lineaTx = data_lineaTx.assign(LinName = linea)
        data_lineaTx.to_json(line_flow_percentil_path+f"/line_{idlin}.json",orient='records')

percentilFL()

Procesando datos de AJahuel110->Sauzal110_BP1 [1/282]
Procesando datos de AJahuel154->Paine154 [2/282]
Procesando datos de AJahuel154->Tuniche154_II [3/282]
Procesando datos de AJahuel220->AJahuel110 [4/282]
Procesando datos de AJahuel220->AJahuel154 [5/282]
Procesando datos de AJahuel220->Buin110 [6/282]
Procesando datos de AJahuel220->Chena220 [7/282]
Procesando datos de AJahuel220->PAltoCmpc110 [8/282]
Procesando datos de AJahuel220->SantaMarta220 [9/282]
Procesando datos de AJahuel500->AJahuel220 [10/282]
Procesando datos de Alfalfal220->Almendros220 [11/282]
Procesando datos de Almendros110->Apoquindo110 [12/282]
Procesando datos de Almendros220->AJahuel220 [13/282]
Procesando datos de Almendros220->Almendros110 [14/282]
Procesando datos de AMelipill220->LoAguirre220 [15/282]
Procesando datos de Ancoa220->Itahue154 [16/282]
Procesando datos de Ancoa500->AJahuel500 [17/282]
Procesando datos de Ancoa500->Ancoa220 [18/282]
Procesando datos de Andes220->Oeste220 [19/282]
Procesando da

# Scenarios

## Creando scenarios

### Bus

In [26]:
# Bus contiene:
'''
		(*) id <int>: identificador de la barra 
		(*) time <int>: instante de registro
		(*) name <str>: nombre de la barra
		marginal_cost <float>: costo marginal, genera el gráfico de costo
					[USD/MWh]
		DemBarE <float>: construye el gráfico de demanda de Energía [MWh]
		DemBarP <float>: construye el gráfico de demanda de Potencia [MW]
		Value <float>: mismo valor que marginal_cost [MWh]
'''

def busscenariofunction(dfbusauxlist, pathbus):
    for x in range(nbus): 
        idbus = indexbus['id'][x]
        aux = pd.DataFrame({
            'id': idbus,
            'time': dfbusauxlist[x]['time'],
            'name': indexbus['BarName'][x],
            'marginal_cost': dfbusauxlist[x]['CMgBar'],
            'value': dfbusauxlist[x]['CMgBar'],
            'DemBarE': dfbusauxlist[x]['DemBarE'],
            'DemBarP': dfbusauxlist[x]['DemBarP'],
            'BarRetP': dfbusauxlist[x]['BarRetP']
        })
        aux.to_json(pathbus + f"/bus_{idbus}.json", orient='records')


for hidronum,hidroname in enumerate(hidrolist):
	
	dfbussauxx=plpbar.query(f"(Hidro=='{hidroname}')").reset_index()
	dfbuslist=[]
	for idaux in lbus:
		dfbuslist.append(dfbussauxx[dfbussauxx.id==idaux].reset_index(drop=True))
	print(f"{((hidronum+1)/len(hidrolist))*100}% Completado")
	busscenariofunction(dfbuslist,busscenariolist[hidronum])

100.0% Completado


### Centrals

In [20]:
indexcen

,id,CenName,tipo,bus_id
0,2,CIPRESES,otros,123
1,3,PEHUENCHE,otros,170
2,4,COLBUN,otros,128
3,5,ELTORO,otros,102
4,6,RAPEL,otros,187
...,...,...,...,...
1672,1717,YUNGAY_U1_DIE,otros,113
1673,1718,YUNGAY_U2_DIE,otros,113
1674,1719,YUNGAY_U3_DIE,otros,113
1675,1720,YUNGAY_U4_DIE,otros,113


In [22]:
indexcen[np.isnan(indexcen['bus_id']) ]

,id,CenName,tipo,bus_id


In [23]:
len(indexcen)

1677

In [27]:
# Centrals contiene:
'''
		(*) id <int>: identificador del generador
		(*) time <int>: instante de registro
		(*) bus_id <int>: identificador de la barra a la que se conecta
		(*) name <str>: nombre del generador
		CenPgen <float>: energía generada en el instante time [MW]
		value <float>: mismo valor que CenPgen [MW]
		(?) CenCVar <unknown>: parámetro no identificado
		(?) CenQgen <unknown>: parámetro no identificado
        
'''

def centralscenariofunction(dfcenauxlist, cenpath):
    for x in range(ngen):
        if indexcen['bus_id'][x] == 0 or np.isnan(indexcen['bus_id'][x]):
            continue
        aux_df = pd.DataFrame({
            'id': indexcen['id'][x],
            'time': range(1, time + 1),
            'bus_id': int(indexcen['bus_id'][x]),
            'name': indexcen['CenName'][x],
            'CenPgen': dfcenauxlist[x]['CenPgen'] if len(dfcenauxlist[x]) > 0 else [0]*time,
            'value': dfcenauxlist[x]['CenPgen'] if len(dfcenauxlist[x]) > 0 else [0]*time,
            'CenCVar': dfcenauxlist[x]['CenCVar'] if len(dfcenauxlist[x]) > 0 else [0]*time,
            'CenQgen': dfcenauxlist[x]['CenQgen'] if len(dfcenauxlist[x]) > 0 else [0]*time,
        })
        aux_df.to_json(cenpath + f"/central_{indexcen['id'][x]}.json", orient='records')

for hidronum, hidroname in enumerate(hidrolist):
    dfcensauxx = plpcen.query(f"(Hidro=='{hidroname}')").reset_index()
    dfcenlist = [dfcensauxx[dfcensauxx.id == indexcen['id'][x]].reset_index(drop=True) for x in range(ngen)]
    print(f"{((hidronum + 1) / len(hidrolist)) * 100}% Completado")
    centralscenariofunction(dfcenlist, centralscenariolist[hidronum])

100.0% Completado


### Lines

In [28]:
len(plplin.LinName.unique())

282

In [29]:
'''

        (*) id <int>: identificador de la linea 
		(*) time <int>: instante de registro
		(*) bus_a <int>: identificador de la barra de origen
		(*) bus_b <int>: identificador de la barra de destino
		flow <float>: flujo en el instante time [MW]
		value <float>: mismo valor que flow [MW]    
'''
def linescenariofunction(dflinelist, linpath):
    for x in range(nlin):
        if linesfinal['active'][x] != 1:
            continue
        idaux = linesfinal['id'][x]
        bus_a_id = linesfinal['bus_a'][x]
        bus_b_id = linesfinal['bus_b'][x]
        name = linesfinal['LinName'][x]
        aux_df = pd.DataFrame({
            'id': idaux,
            'time': range(1, time + 1),
            'name': name,
            'bus_a': bus_a_id,
            'bus_b': bus_b_id,
            'flow': dflinelist[x]['LinFluP'],
            'value': dflinelist[x]['LinFluP'],
            'capacity': dflinelist[x]['capacity'],
        })
        aux_df.to_json(linpath + f"/line_{idaux}.json", orient='records')

for hidronum, hidroname in enumerate(hidrolist):
    dflinesaux = plplin.query(f"(Hidro=='{hidroname}')").reset_index()
    dflinelist = [dflinesaux[dflinesaux.id == linesfinal['id'][x]].reset_index(drop=True) for x in range(nlin)]
    print(f"{((hidronum + 1) / len(hidrolist)) * 100}% Completado")
    linescenariofunction(dflinelist, linescenariolist[hidronum])


100.0% Completado


### Reservoirs

In [25]:
# Resevoirs contiene:
'''
		(*) time <int>: instante de registro
		(*) id <int>: identificador del embalse
		(*) junction_id <int>: identificador del canal al que se conecta
		(*) name <str>: nombre del embalse
		level <float>: nivel en el instante time
		value <float>: mismo valor que level
'''


def resscenariofunction(dfreslist, respath):
    for x in range(nres):
        idaux = indexres['id'][x]
        name = indexres['EmbName'][x]
        junction_id = junctionsinfo[junctionsinfo['CenName'] == name]['id'].values[0]
        aux_df = pd.DataFrame({
            'time': range(1, time + 1),
            'id': idaux,
            'junction_id': junction_id,
            'name': name,
            'level': (dfreslist[x]['EmbFac'] * dfreslist[x]['EmbVfin']) / 1000000,
            'value': (dfreslist[x]['EmbFac'] * dfreslist[x]['EmbVfin']) / 1000000,
        })
        aux_df.to_json(respath + f"/reservoir_{idaux}.json", orient='records')

for hidronum, hidroname in enumerate(hidrolist):
    dfresaux = reservoirs.query(f"(Hidro=='{hidroname}')").reset_index()
    dfreslist = [dfresaux[dfresaux.id == indexres['id'][x]].reset_index(drop=True) for x in range(nres)]
    print(f"{((hidronum + 1) / len(hidrolist)) * 100}% Completado")
    resscenariofunction(dfreslist, reservoirscenariolist[hidronum])


NameError: name 'reservoirs' is not defined

# Topology

## Electric

### bus

In [30]:
ubibar[['latUTM','lonUTM']]=ubibar.apply(lambda row: valorXY(row['latitud'],row['longitud'],scale=0.001),axis=1,result_type='expand')
dirdfbus=ubibar

In [31]:
# bus electric contiene:

'''   
		(*) id <int>: identificador de la barra
		(*) name <str>: nombre de la barra
		longitude <float>
		latitude <float>
		active <int>: indica si la barra está activa
'''
auxiliar=[]
buselectricfilas_aux=[]
for x in range(nbus): # Para cada barra (bus)
	if dirdfbus['BarName'].isin([indexbus['BarName'][x]]).tolist().count(True)>0:
		latitud=float(dirdfbus[dirdfbus['BarName']==indexbus['BarName'][x]]['latitud'].values[0])
		longitud=float(dirdfbus[dirdfbus['BarName']==indexbus['BarName'][x]]['longitud'].values[0])
	else:
		auxiliar.append(indexbus['BarName'][x])
		latitud,longitud=aleatory_direction()

	aux=[]
	aux.append(indexbus['id'][x])
	aux.append(indexbus['BarName'][x])
	aux.append(longitud)
	aux.append(latitud)
	aux.append(1)
	buselectricfilas_aux.append(aux)

buselectric=pd.DataFrame(buselectricfilas_aux,columns=['id','name','longitude','latitude','active'])

buselectric.to_json(electricTopology+"/bus.json",orient='records')


### centrals

In [48]:
# centrals electric contiene:

'''   
        (*) id <int>: identificador del generador
		(*) bus_id <int>: id de la barra conectada al generador
		(*) name <str>: nombre del generador
		active <int>: indica si el generador está activo
		capacity <float>: capacidad del generador [MW]
		min_power <float>: generación mínima [MW]
		max_power <float>: generación máxima [MW]
		type <str>: tipo de generador
		longitude <float>
		latitude <float>
		(?) effinciency <float>: Rendimiento [MWh/m3s]
		(?) flow <float>: parámetro no identificado
		(?) rmin <float>: parámetro no identificado
		(?) rmax <float>: parámetro no identificado
		(?) cvar <float>: Costo Variable
		(?) cvnc <unknown>: parámetro no identificado
		(?) cvc <unknown>: parámetro no identificado
		(?) entry_date <unknown>: parámetro no identificado

'''

centralselectricfilas_aux=[]
for x in range(ngen): # Para cada generador (central)
	if indexcen['bus_id'][x]==0 or np.isnan(indexcen['bus_id'][x]): # No existe la barra 0, por lo que no se consideran dichos generadores
		pass
	else:
		latitud,longitud=None,None
		aux=[]
		aux.append(indexcen['id'][x])
		aux.append(int(indexcen['bus_id'][x]))
		aux.append(indexcen['CenName'][x])
		aux.append(1)
		# capacidad
		aux.append(0)
		aux.append(centralsinfo[centralsinfo['CenName']==indexcen['CenName'][x]]['min_power'])
		aux.append(centralsinfo[centralsinfo['CenName']==indexcen['CenName'][x]]['max_power'])
		tipo=typecentrals[typecentrals['CenName']==indexcen['CenName'][x]]['cen_type'].values
		if len(tipo)>0:
			aux.append(tipo[0])
		else:
			aux.append("otros")
		aux.append(longitud)
		aux.append(latitud)
		aux.append(centralsinfo[centralsinfo['CenName']==indexcen['CenName'][x]]['effinciency'])
		for x in range(7):
			aux.append(0)
		centralselectricfilas_aux.append(aux)

centralelectric=pd.DataFrame(centralselectricfilas_aux,columns=['id','bus_id','name','active','capacity','min_power','max_power','type','longitude','latitude','efficiency','flow','rmin','rmax','cvar',
'cvnc','cvc','entry_date'])

centralelectric.to_json(electricTopology+"/centrals.json",orient='records')

### lines

In [33]:
# Lines electric tiene:
'''  
        (*) id <int>: identificador de la línea
		(*) bus_a <int>: id de la barra origen
		(*) bus_b <int>: id de la barra destino
		active <int>: indica si la línea está activa
		capacity <float>: capacidad máxima de la línea [MW]  ->
		max_flow_a_b <float>: flujo máximo en dirección
					dispuesta [MW]
		max_flow_b_a <float>: flujo máximo en dirección
					contraria [MW]
		voltage <float>: voltaje de la línea [kV]
		r <float>: resistencia de la línea [Ω]
		x <float>: reactancia de la línea [Ω]
		(? )segments <int>: parámetro no identificado
		(?) entry_date <unknown>: parámetro no identificado
		(?) exit_date <unknown>: parámetro no identificado

'''

lineselectricfilas_aux=[]
for x in range(nlin): # Para cada linea
	if linesfinal['active'][x]==1:
		aux=[]
		bus_a_id = linesfinal['bus_a'][x]
		bus_b_id = linesfinal['bus_b'][x]
		name = linesfinal['LinName'][x]
		aux.append(linesfinal['id'][x])
		aux.append(name)
		aux.append(bus_a_id)
		aux.append(bus_b_id)
		aux.append(1)
		# capacidad
		aux.append(0)
		aux.append(linesfinal['max_flow_a_b'][x])
		aux.append(linesfinal['max_flow_b_a'][x])
		aux.append(linesfinal['voltage'][x])
		aux.append(linesfinal['r'][x])
		aux.append(linesfinal['x'][x])
		aux.append(linesfinal['segments'][x])
		aux.append(None)
		aux.append(None)
		lineselectricfilas_aux.append(aux)

lineelectric=pd.DataFrame(lineselectricfilas_aux,columns=['id','name','bus_a','bus_b','active','capacity','max_flow_a_b','max_flow_b_a','voltage','r','x','segments','entry_date','exit_date'])

lineelectric.to_json(electricTopology+"/lines.json",orient='records')

## Hydric

### Reservoirs

In [ ]:
'''   
        (*) id <int>: identificador del embalse
		(*) junction_id <int>: id del embalse relacionada (mismo valor id)
		(*) name <str>: nombre del embalse
		(*) type <str>: tipo de embalse
		min_vol <float>: volumen mínimo del embalse
		max_vol <float>: volumen máximo del embalse
		start_vol <float>: volumen inicial del embalse
		end_vol <float>: volumen final del embalse
		active <bool>: indica si el embalse está activo
		(?) hyd_independant <bool>: parámetro no identificado
		(?) future_cost <unknown>: parámetro no identificado
		(?) cmin <unknown>: cota m.s.n.m mínima


'''


reshydricfilas_aux=[]
for x in range(nres): # Para cada linea
	aux=[]
	idaux=indexres['id'][x]
	name=indexres['EmbName'][x]
	junction_id = junctionsinfo[junctionsinfo['CenName']==name]['id'].values[0]
	
	aux.append(idaux)
	aux.append(junction_id)
	aux.append(name)
	aux.append(reservoirsinfo[reservoirsinfo['id']==idaux]['type'].values[0])
	aux.append(reservoirsinfo[reservoirsinfo['id']==idaux]['VembMin'].values[0])
	aux.append(reservoirsinfo[reservoirsinfo['id']==idaux]['VembMax'].values[0])
	aux.append(reservoirsinfo[reservoirsinfo['id']==idaux]['VembIn'].values[0])
	aux.append(reservoirsinfo[reservoirsinfo['id']==idaux]['VembFin'].values[0])
	aux.append(1)
	aux.append(0)
	aux.append(None)
	aux.append(reservoirsinfo[reservoirsinfo['id']==idaux]['cotaMínima'].values[0])
	reshydricfilas_aux.append(aux)

reshydric=pd.DataFrame(reshydricfilas_aux,columns=['id','junction_id','name','type','min_vol','max_vol','start_vol','end_vol','active','hyd_independant','future_cost','cmin'])

reshydric.to_json(hydricTopology+"/reservoirs.json",orient='records')

### Junctions

In [30]:
reservoirsinfo

,id,EmbName,type,CVar,effinciency,bus_id,serie_hidro_gen,serie_hidro_ver,min_power,max_power,VembIn,VembFin,VembMin,VembMax,cotaMínima
0,1,LMAULE,E,NaN,1,0.0,2.0,NaN,0,100,351.5262388,1293.459972,0,1453.409259,2152.1
1,2,CIPRESES,E,NaN,2.878570106,123.0,9.0,9.0,0,105,110.4410089,160.5394291,4.716356886,174.6603258,1282.8
2,3,PEHUENCHE,E,NaN,1.78,170.0,25.0,22.0,0,550,120.2934857,121.6159275,106.5830731,133.6440504,641
3,4,COLBUN,E,NaN,1.53,128.0,33.0,NaN,0,474,1183.820764,1553.245913,381.6242997,1553.245913,397
4,5,ELTORO,E,NaN,4.8,102.0,39.0,38.0,0,450,1327.788248,3777.43006,0,5585.88761,1300
5,6,RAPEL,E,NaN,0.64,187.0,NaN,NaN,0,375,478.5838602,563.2124335,272.3049253,563.2124335,100.5
6,7,CANUTILLAR,E,NaN,2,110.0,NaN,NaN,0,170.2,745.897608,1065.3785,449.739,1065.3785,230
7,8,RALCO,E,NaN,1.657333,185.0,67.0,67.0,0,690,927.8745121,756.9598992,409.4096136,1173.286179,692
8,9,PANGUE,E,NaN,0.8,167.0,68.0,68.0,80,450,69.71282456,69.71282456,57.5834,71.96,507
9,10,PILMAIQUEN,E,NaN,0.27,184.0,84.0,84.0,7,39,132.42528,89.387064,0,289.6803,102


In [41]:
junctionsinfo[junctionsinfo['bus_id']== 0]

,id,CenName,type,CVar,effinciency,bus_id,serie_hidro_gen,serie_hidro_ver,min_power,max_power,VembIn,VembFin,VembMin,VembMax,cotaMínima
0,1,LMAULE,E,NaN,1,0.0,2.0,NaN,0,100,351.5262388,1293.459972,0,1453.409259,2152.1
2,3,B_LaMina,S,NaN,1,0.0,5.0,4.0,0,6.96,NaN,NaN,NaN,NaN,NaN
4,5,B_M_Isla,S,NaN,1,0.0,10.0,13.0,0,48,NaN,NaN,NaN,NaN,NaN
6,7,FILT_CIPRESES,S,NaN,1,0.0,8.0,8.0,0,20.1,NaN,NaN,NaN,NaN,NaN
8,9,B_C_Isla,S,NaN,1,0.0,10.0,13.0,0,50,NaN,NaN,NaN,NaN,NaN
12,13,B_Maule,S,NaN,1,0.0,21.0,22.0,0,140,NaN,NaN,NaN,NaN,NaN
13,14,Rio_Melado,S,NaN,1,0.0,15.0,21.0,0,25.3,NaN,NaN,NaN,NaN,NaN
14,15,C_Melado,R,NaN,1,0.0,20.0,16.0,0,19,NaN,NaN,NaN,NaN,NaN
19,20,RieMelado,R,NaN,1,0.0,NaN,NaN,0,9999,NaN,NaN,NaN,NaN,NaN
21,22,CMNTronco,R,NaN,1,0.0,23.0,25.0,0,9999,NaN,NaN,NaN,NaN,NaN


In [31]:
'''
	(*) id <int>: identificador de la unión
	(*) name <str>: nombre de la unión
	longitude <float>
	latitude <float>
	active <bool>: indica si la barra está activa
	drainage <bool>: parámetro no identificado
	bus_id

'''

junctionhydricfilas_aux=[]
for x in range(len(junctionsinfo)): # Para cada junction
	latitud,longitud=aleatory_direction()
	aux=[]
	aux.append(junctionsinfo['id'][x])
	aux.append(junctionsinfo['CenName'][x])
	aux.append(longitud)
	aux.append(latitud)
	aux.append(1)
	aux.append(0)
	aux.append(junctionsinfo['bus_id'][x])
	
	junctionhydricfilas_aux.append(aux)

junctionhydric=pd.DataFrame(junctionhydricfilas_aux,columns=['id','name','logitude','latitude','active','drainage','bus_id'])

junctionhydric.to_json(hydricTopology+"/junctions.json",orient='records')

### Waterways

In [ ]:
'''
        (*) id <int>: identificador del canal
		(*) name <str>: nombre del canal
		(*) type <str>: tipo de waterway
		(*) junc_a_id <int>: id de la unión de origen
		(*) junc_b_id <int>: id de la unión de destino
		active <bool>: indica si el canal está activo
		(?) fmin <unknown>: parámetro no identificado
		(?) fmax <unknown>: parámetro no identificado
		(?) cvar <unknown>: parámetro no identificado 
        (?) delay <unknown>: parámetro no identificado

'''

junctionhydricfilas_aux=[]
countid=1
for x in range(len(junctionsinfo)):
    gen_id=junctionsinfo.serie_hidro_gen[x]
    ver_id=junctionsinfo.serie_hidro_ver[x]
    name_a = junctionsinfo.CenName[x]
    df_adicional = hydric_adicional[hydric_adicional['embalse'] == name_a]
    if not pd.isnull(gen_id):
        aux=[]
        aux.append(countid)
        countid+=1
        name_b = junctionsinfo[junctionsinfo['id']==gen_id].CenName.values[0]
        name = name_a+'_Gen_'+name_b
        aux.append(name)
        aux.append("generation")
        aux.append(junctionsinfo.id[x])
        aux.append(gen_id)
        aux.append(1)
        aux.append(None)
        aux.append(None)
        aux.append(None)
        aux.append(None)
        junctionhydricfilas_aux.append(aux)

    if not pd.isnull(ver_id):
        aux=[]
        aux.append(countid)
        countid+=1
        name_b = junctionsinfo[junctionsinfo['id']==ver_id].CenName.values[0]
        name = name_a+'_Vert_'+name_b
        aux.append(name)
        aux.append("spillover")
        aux.append(junctionsinfo.id[x])
        aux.append(ver_id)
        aux.append(1)
        aux.append(None)
        aux.append(None)
        aux.append(None)
        aux.append(None)
        junctionhydricfilas_aux.append(aux)
    if len(df_adicional)>0:
        for i in range(len(df_adicional)):
            tipo =df_adicional['type'].iloc[i]
            name =""
            central = df_adicional['central'].iloc[i].lower()
            id_central = centralsinfo[centralsinfo['CenName'].str.lower() == central]['id'].values[0]
            aux=[]
            aux.append(countid)
            countid+=1
            name_b = junctionsinfo[junctionsinfo['id']==id_central].CenName.values[0]
            if tipo == "filtration":
                name = name_a+'_Fil_'+name_b
            elif tipo == "extraction":
                name = name_a+'_Ext_'+name_b
            aux.append(name)
            aux.append(tipo)
            aux.append(junctionsinfo.id[x])
            aux.append(id_central)
            aux.append(1)
            aux.append(None)
            aux.append(None)
            aux.append(None)
            aux.append(None)
            junctionhydricfilas_aux.append(aux)
waterwayshydric=pd.DataFrame(junctionhydricfilas_aux,columns=["id","name","type","junc_a_id","junc_b_id","active","fmin","fmax","cvar","delay"])
waterwayshydric.to_json(hydricTopology+"/waterways.json",orient='records')